# **Introduction:**

This file serves to perform further grid searching to further determine the best hyperparameters for an ANN model for use in multi-robot task allocation through regression on FIS-generated data. The goal for designing this ANN is to compare its performance against an ANFIS to determine which is better at approximating the FIS, which will be achieved through using the coefficient of determination ($R^{2}$), root mean squared error (RMSE), and mean absolute error (MAE).

**Date Created:** 17/12/2024

**Date Modified:** 18/12/2024

# **Import Packages:**

This section imports all necessary packages for the ANN implementation.

In [1]:
# import packages:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import json

# **Data Loading:**

This section loads the data that was generated from the FIS. Minimal discovery is performed here, as the bulk of the data discovery was performed within the first grid search.

In [2]:
# get the path to the data CSV:
files_in_dir = os.listdir(os.getcwd())
data_path = os.path.join(os.getcwd(), files_in_dir[files_in_dir.index('V3_Data.csv')])

# load the CSV as a pandas dataframe:
df = pd.read_csv(data_path)
print(f"data successfully loaded")

data successfully loaded


# **Data Pre-Processing:**

This section will split the data into training, validation, and testing, alongside performing some pre-processing.

In [3]:
# get feature and label dataframes:
x_data = df.drop(['Suitability'], axis = 1)
y_data = df['Suitability']

Need to first standardize the values:

In [4]:
# define scaler:
scaler = StandardScaler()
x_data_scaled = scaler.fit_transform(x_data)

Split data into training, validation, and testing:

In [5]:
# split dataset:
x_train, x_test, y_train, y_test = train_test_split(x_data_scaled, y_data, test_size = 0.2)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size = 0.5)

# get split results:
print(f"there are {x_train.shape[0]} training examples")
print(f"there are {x_val.shape[0]} validation examples")
print(f"there are {x_test.shape[0]} testing examples")

# get input shape:
INPUT_SHAPE = x_data.shape[1]

there are 12000 training examples
there are 1500 validation examples
there are 1500 testing examples


# **Model Exploration:**

Within this section, a function is defined for instantiating models using the Keras API, for use in performing a hyperparameter search to determine the best combination of hyperparameters. The hyperparameters that are being considered are:

* number of hidden layers
* number of hidden neurons
* batch size

In [6]:
# query the user for whether they want to grid search or not:
grid_search = input('perform grid search? (True/False)')

if grid_search.strip().lower() == 'true':
    grid_search = True

    num_hidden_layers = [2,3,4]
    num_neurons = [64, 80, 96, 112, 128]
    batch_sizes = [128, 160, 192, 224, 256]

    combinations = len(num_hidden_layers) * len(num_neurons) * len(batch_sizes)
    LOSS_FUNCTION = 'mse'
    METRICS = ['mae', keras.metrics.RootMeanSquaredError(), keras.metrics.R2Score]
else: 
    grid_search == False

Define model generation function:

In [7]:
def make_model(layers, neurons, rate, norm, drop):
    # instantiate model:
    model = keras.Sequential()

    # add hidden layers:
    for i in range(layers):
        if i == 0:
            model.add(Input(shape = (INPUT_SHAPE, )))
            model.add(Dense(neurons, activation = 'relu', name = f'hidden_layer_{i+1}'))
        else:
            model.add(Dense(neurons, activation = 'relu', name = f'hidden_layer_{i+1}'))
        
        if norm == True:
            model.add(BatchNormalization(name = f'batch_norm_layer_{i+1}'))
        
        if drop == True:
            model.add(Dropout(0.2, name = f'dropout_layer_{i+1}'))
        
    # add output layer:
    model.add(Dense(1, activation = 'linear', name = 'output_layer'))

    # compile the model:
    model.compile(optimizer = Adam(learning_rate = rate),
                  loss = LOSS_FUNCTION,
                  metrics = METRICS)
    
    return model

Now we must perform the grid search. This process entails:

* Creating a directory to save the search results in
* Creating a model using the aforementioned "make_model()" function
* Save the parameters used in the creation of the model within dictionary called "model_params"
* Train the model, saving results into a dictionary called "training results"
* Combining the training parameters with the training results into a JSON dump

While iterating through each combination of parameters.

In [8]:
if grid_search == True:
    j = 1
    # set up the grid search:
    for layer in num_hidden_layers:
        for neurons in num_neurons:
            for batch in batch_sizes:
                # update the user:
                print(f"examining model {j}/{combinations}", end = '\r')
                j+=1

                # make directory to save into:
                output_dir = os.path.join(os.getcwd(), f"ann_search_results//{str(layer)}_{str(neurons)}_{str(batch)}")
                os.makedirs(output_dir, exist_ok = True)

                # build a model:
                tf.keras.backend.clear_session()
                model = make_model(layer, neurons, 0.001, True, True)

                # save training parameters into a dictionary:
                training_params = {
                    'num_layers' : layer,
                    'num_neurons' : neurons,
                    'num_epochs' : 500,
                    'learning_rate' : 0.001,
                    'use_batch_norm' : True,
                    'use_dropout' : True,
                    'batch_size' : batch
                }

                # train model:
                train_start = time.time()
                history = model.fit(x_train, y_train,
                                    epochs = 500,
                                    batch_size = batch,
                                    validation_data = [x_val, y_val],
                                    verbose = 0)
                
                train_time = time.time() - train_start

                # store training results:
                training_results = {}
                for i in history.history.keys():
                    training_results[i] = history.history[i][-1]
                training_results['train_time'] = train_time

                # save both results to a directory:
                params_path = os.path.join(output_dir, 'params_results.json')
                with open(params_path, "w") as f:
                    json.dump({'parameters': training_params, 'results': training_results}, f, indent = 4)         

# **Examine Hyperparameter Search Results:**

This section examines the data that was collected during the hyperparameter grid search. Each combination of hyperparameters had its training parameters and training results saved into separate dictionaries, which where then concatenated into a JSON dump. This section pertains to iterating through each of the folders of the tests and amalgamating the results into a Pandas DataFrame for further analysis:

In [9]:
# initialize results list:
results = []
grid_search_folder = os.path.join(os.getcwd(), 'ann_search_results')

for folder in os.listdir(grid_search_folder):
    folder_path = os.path.join(grid_search_folder, folder)
    if os.path.isdir(folder_path):
        params_file = os.path.join(folder_path, 'params_results.json')

        if os.path.exists(params_file):
            with open(params_file, 'r') as f:
                data = json.load(f)

                # flatten the JSON:
                extracted_data = {
                    # training params:
                    'num_layers' : data['parameters']['num_layers'],
                    'num_neurons' : data['parameters']['num_neurons'],
                    'num_epochs' : data['parameters']['num_epochs'],
                    'learning_rate' : data['parameters']['learning_rate'],
                    'batch_size' : data['parameters']['batch_size'],

                    # training results:
                    'train_loss' : data['results']['loss'],
                    'train_mae' : data['results']['mae'],
                    'train_rmse' : data['results']['root_mean_squared_error'],
                    'training_r2' : data['results']['r2_score'],
                    'val_loss' : data['results']['val_loss'],
                    'val_mae' : data['results']['val_mae'],
                    'val_rmse' : data['results']['val_root_mean_squared_error'],
                    'val_r2' : data['results']['val_r2_score'],
                    'training_time' : data['results']['train_time']
                }

                results.append(extracted_data)

# turn the results into a dataframe:
results_df = pd.DataFrame(results)

# insert an identifier for models:
results_df.insert(0, 'model_name', [f'model {index + 1}' for index, row in results_df.iterrows()])

# save consolidated data into a CSV file:
results_df.to_csv('consolidated_results.csv', index = False)

Need to now determine the best hyperparameter combination based on the results from this analysis, which have been consolidated into a single DataFrame. Going to organize the DataFrame by the best of each metric. The metrics that will be examined are:

* Training MSE (loss)
* Validation MSE (loss)
* Validation MAE
* Validation RMSE
* Validation $R^{2}$

Sort the consolidated results by the lowest training loss:

In [10]:
results_df.sort_values(by = 'train_loss', ascending = True).head(15)

,model_name,num_layers,num_neurons,num_epochs,learning_rate,batch_size,train_loss,train_mae,train_rmse,training_r2,val_loss,val_mae,val_rmse,val_r2,training_time
58,model 59,4,128,500,0.001,224,0.043347,0.158455,0.208200,0.980041,0.005008,0.051598,0.070769,0.997765,69.466295
28,model 29,3,112,500,0.001,224,0.046119,0.162221,0.214753,0.978765,0.010894,0.086392,0.104373,0.995139,52.416141
7,model 8,2,128,500,0.001,192,0.046758,0.166362,0.216236,0.978470,0.006232,0.056633,0.078943,0.997219,46.512810
3,model 4,2,112,500,0.001,224,0.047151,0.165472,0.217143,0.978289,0.008740,0.071800,0.093487,0.996100,42.048487
51,model 52,4,112,500,0.001,160,0.047364,0.166168,0.217633,0.978191,0.008380,0.069358,0.091543,0.996261,66.518150
33,model 34,3,128,500,0.001,224,0.047438,0.166999,0.217803,0.978157,0.013404,0.090126,0.115776,0.994019,56.634217
57,model 58,4,128,500,0.001,192,0.047689,0.166813,0.218379,0.978042,0.007087,0.064955,0.084187,0.996837,70.165697
56,model 57,4,128,500,0.001,160,0.047747,0.167481,0.218511,0.978015,0.019412,0.119518,0.139327,0.991338,69.178798
5,model 6,2,128,500,0.001,128,0.047980,0.168672,0.219043,0.977908,0.005409,0.051562,0.073543,0.997587,55.102768
9,model 10,2,128,500,0.001,256,0.048255,0.168899,0.219671,0.977781,0.013608,0.085129,0.116654,0.993928,42.180613


Sort the consolidated results by lowest validation loss:

In [11]:
results_df.sort_values(by = 'val_loss', ascending = True).head(15)

,model_name,num_layers,num_neurons,num_epochs,learning_rate,batch_size,train_loss,train_mae,train_rmse,training_r2,val_loss,val_mae,val_rmse,val_r2,training_time
26,model 27,3,112,500,0.001,160,0.055363,0.181414,0.235294,0.974508,0.004875,0.050125,0.069818,0.997825,55.973476
29,model 30,3,112,500,0.001,256,0.049657,0.169169,0.222838,0.977136,0.005003,0.050496,0.070732,0.997768,49.773012
58,model 59,4,128,500,0.001,224,0.043347,0.158455,0.208200,0.980041,0.005008,0.051598,0.070769,0.997765,69.466295
53,model 54,4,112,500,0.001,224,0.052520,0.173649,0.229172,0.975817,0.005176,0.050196,0.071946,0.997690,62.350225
5,model 6,2,128,500,0.001,128,0.047980,0.168672,0.219043,0.977908,0.005409,0.051562,0.073543,0.997587,55.102768
32,model 33,3,128,500,0.001,192,0.050196,0.172096,0.224045,0.976887,0.005456,0.053593,0.073865,0.997565,58.789856
73,model 74,4,96,500,0.001,224,0.056867,0.181340,0.238469,0.973816,0.005661,0.056091,0.075242,0.997474,55.933677
47,model 48,3,96,500,0.001,192,0.051519,0.173407,0.226978,0.976278,0.005729,0.054737,0.075691,0.997444,50.906558
1,model 2,2,112,500,0.001,160,0.050968,0.174327,0.225760,0.976532,0.005860,0.055491,0.076551,0.997385,45.321052
48,model 49,3,96,500,0.001,224,0.055507,0.178335,0.235599,0.974442,0.006049,0.055806,0.077774,0.997301,48.558777


Sort the consolidated results by the lowest validation MAE:

In [12]:
results_df.sort_values(by = 'val_mae', ascending = True).head(5)

,model_name,num_layers,num_neurons,num_epochs,learning_rate,batch_size,train_loss,train_mae,train_rmse,training_r2,val_loss,val_mae,val_rmse,val_r2,training_time
26,model 27,3,112,500,0.001,160,0.055363,0.181414,0.235294,0.974508,0.004875,0.050125,0.069818,0.997825,55.973476
53,model 54,4,112,500,0.001,224,0.052520,0.173649,0.229172,0.975817,0.005176,0.050196,0.071946,0.997690,62.350225
29,model 30,3,112,500,0.001,256,0.049657,0.169169,0.222838,0.977136,0.005003,0.050496,0.070732,0.997768,49.773012
5,model 6,2,128,500,0.001,128,0.047980,0.168672,0.219043,0.977908,0.005409,0.051562,0.073543,0.997587,55.102768
58,model 59,4,128,500,0.001,224,0.043347,0.158455,0.208200,0.980041,0.005008,0.051598,0.070769,0.997765,69.466295


Sort the consolidated results by the lowest validation RMSE:

In [13]:
results_df.sort_values(by = 'val_rmse', ascending = True).head(5)

,model_name,num_layers,num_neurons,num_epochs,learning_rate,batch_size,train_loss,train_mae,train_rmse,training_r2,val_loss,val_mae,val_rmse,val_r2,training_time
26,model 27,3,112,500,0.001,160,0.055363,0.181414,0.235294,0.974508,0.004875,0.050125,0.069818,0.997825,55.973476
29,model 30,3,112,500,0.001,256,0.049657,0.169169,0.222838,0.977136,0.005003,0.050496,0.070732,0.997768,49.773012
58,model 59,4,128,500,0.001,224,0.043347,0.158455,0.208200,0.980041,0.005008,0.051598,0.070769,0.997765,69.466295
53,model 54,4,112,500,0.001,224,0.052520,0.173649,0.229172,0.975817,0.005176,0.050196,0.071946,0.997690,62.350225
5,model 6,2,128,500,0.001,128,0.047980,0.168672,0.219043,0.977908,0.005409,0.051562,0.073543,0.997587,55.102768


Sort the consolidated results by the highest validation $R^{2}$:

In [14]:
results_df.sort_values(by = 'val_r2', ascending = False).head(5)

,model_name,num_layers,num_neurons,num_epochs,learning_rate,batch_size,train_loss,train_mae,train_rmse,training_r2,val_loss,val_mae,val_rmse,val_r2,training_time
26,model 27,3,112,500,0.001,160,0.055363,0.181414,0.235294,0.974508,0.004875,0.050125,0.069818,0.997825,55.973476
29,model 30,3,112,500,0.001,256,0.049657,0.169169,0.222838,0.977136,0.005003,0.050496,0.070732,0.997768,49.773012
58,model 59,4,128,500,0.001,224,0.043347,0.158455,0.208200,0.980041,0.005008,0.051598,0.070769,0.997765,69.466295
53,model 54,4,112,500,0.001,224,0.052520,0.173649,0.229172,0.975817,0.005176,0.050196,0.071946,0.997690,62.350225
5,model 6,2,128,500,0.001,128,0.047980,0.168672,0.219043,0.977908,0.005409,0.051562,0.073543,0.997587,55.102768


Sort the consolidated results by lowest training loss, validation loss, validation MAE, validation RMSE, and highest validation $R^{2}$:

In [15]:
results_df.sort_values(by = ['train_loss', 'val_loss', 'val_mae', 'val_rmse', 'val_r2'], ascending = [True, True, True, True, False]).head(3)

,model_name,num_layers,num_neurons,num_epochs,learning_rate,batch_size,train_loss,train_mae,train_rmse,training_r2,val_loss,val_mae,val_rmse,val_r2,training_time
58,model 59,4,128,500,0.001,224,0.043347,0.158455,0.208200,0.980041,0.005008,0.051598,0.070769,0.997765,69.466295
28,model 29,3,112,500,0.001,224,0.046119,0.162221,0.214753,0.978765,0.010894,0.086392,0.104373,0.995139,52.416141
7,model 8,2,128,500,0.001,192,0.046758,0.166362,0.216236,0.978470,0.006232,0.056633,0.078943,0.997219,46.512810


# **Results & Conclusions - Grid Search V2:**

From this analysis it can be seen that the best performing model was that of ***model 59***, which had four layers, each with 128 neurons, and was trained for 500 epochs using a learning rate of 0.001 and a batch size of 224. This model had the lowest training MSE, meaning that if performed quite well on the dataset provided, and seeing as the validation loss was lower than the training loss, it was able to generalize to the unseen data.

When comparing between the models, their metrics can be examined:

* **Training MSE:** it was found that the best model from the first grid search had a training MSE of 0.068534, whereas the best model from the second grid search had a training MSE of 0.043347, resulting in a **reduction of 36.75%**.

* **Validation MSE:** it was found that the best model from the first grid search had a validation MSE of 0.011547, whereas the best model from the second grid search had a validation MSE of 0.004875, resulting in a **reduction of 57.78%**.

* **Validation MAE:** it was found that the best model from the first grid search had a validation MAE of 0.079588, whereas the best model from the second grid search had a validation MAE of 0.050125, resulting in a **reduction of 37.02%**.

* **Validation RMSE:** it was found that the best model from the first grid search had a validation RMSE of 0.107457, whereas the best model from the second grid search had a validation RMSE of 0.069818, resulting in a **reduction of 35.03%**.

* **Valdation $R^{2}$:** it was found that the best model from the first grid search had a validation $R^{2}$ value of 0.994909, whereas the best model from the second grid search had a validation $R^{2}$ value of 0.997825, resulting in a difference of $2.922$ x $10^{-3}$.

Therefore, the best model that has been determined thus far is ***model 59***, which boasts impressive reductions in MSE, MAE, and RMSE compared to its predecessors. These hyperparameters will be chosen for the ANN trained on FIS data:

* 4 layers
* 128 neurons per layer
* trained for 500 epochs
* trained with a learning rate of 0.001
* trained with a batch size of 224